# Pirelli F1 Tire Data Web Scraper

This notebook scrapes official tire and circuit data from Pirelli's website to complement F1 tire degradation analysis.

## Target Data:
- **Tire Compounds**: Soft/Medium/Hard compound specifications (C1-C5)
- **Circuit Length**: Track distance in km
- **Track Characteristics** (1-5 scale):
  - Traction
  - Asphalt Grip
  - Tire Stress
  - Braking
  - Lateral Forces
  - Downforce
  - Asphalt Abrasion
  - Track Evolution

## Output:
Structured DataFrame/CSV with all circuit data for analysis integration.

## Setup and Imports

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from urllib.parse import urljoin, urlparse
import json
from typing import Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')

print("🕷️ Pirelli Data Scraper Ready!")
print("📊 Target: Tire compounds and circuit characteristics (2022-2024)")

🕷️ Pirelli Data Scraper Ready!
📊 Target: Tire compounds and circuit characteristics (2022-2024)


## Scraper Configuration

In [19]:
# Configuration
BASE_URL = "https://www.pirelli.com/global/en-ww/emotions-and-numbers/"
YEARS = [2022, 2023, 2024]
DELAY_BETWEEN_REQUESTS = 1  # Seconds - be respectful to the server

# Headers to mimic a real browser
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}

print(f"🎯 Target years: {YEARS}")
print(f"⏱️ Delay between requests: {DELAY_BETWEEN_REQUESTS}s")

🎯 Target years: [2022, 2023, 2024]
⏱️ Delay between requests: 1s


## Helper Functions

In [38]:
def get_page_content(url: str) -> Optional[BeautifulSoup]:
    """
    Safely fetch and parse a web page.
    
    Args:
        url: URL to fetch
        
    Returns:
        BeautifulSoup object or None if failed
    """
    try:
        print(f"📡 Fetching: {url}")
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        time.sleep(DELAY_BETWEEN_REQUESTS)  # Be respectful
        return soup
        
    except requests.RequestException as e:
        print(f"❌ Error fetching {url}: {e}")
        return None


def extract_tire_compounds(soup: BeautifulSoup) -> Dict[str, str]:
    """
    Extract tire compound information from Pirelli race pages.
    
    Args:
        soup: Parsed HTML content
        
    Returns:
        Dictionary with tire compound info
    """
    compounds = {'soft': None, 'medium': None, 'hard': None}
    
    try:
        page_text = soup.get_text()
        
        # Enhanced patterns for tire compounds
        compound_patterns = [
            # Pattern: "Soft: C3" or "C3 (Soft)"
            (r'soft[:\s\-]*C([1-5])', 'soft'),
            (r'medium[:\s\-]*C([1-5])', 'medium'), 
            (r'hard[:\s\-]*C([1-5])', 'hard'),
            # Pattern: "C3 (Soft)" or "C3 - Soft"
            (r'C([1-5])[:\s\-\(]*soft', 'soft'),
            (r'C([1-5])[:\s\-\(]*medium', 'medium'),
            (r'C([1-5])[:\s\-\(]*hard', 'hard'),
        ]
        
        for pattern, compound_type in compound_patterns:
            matches = re.findall(pattern, page_text, re.IGNORECASE)
            if matches and not compounds[compound_type]:  # Only set if not already found
                compounds[compound_type] = f"C{matches[0]}"
        
        # Alternative: Look for compounds in image alt text or data attributes
        images = soup.find_all('img')
        for img in images:
            alt_text = img.get('alt', '').lower()
            src = img.get('src', '').lower()
            
            # Check for compound info in image metadata
            for text in [alt_text, src]:
                for pattern, compound_type in compound_patterns:
                    matches = re.findall(pattern, text, re.IGNORECASE)
                    if matches and not compounds[compound_type]:
                        compounds[compound_type] = f"C{matches[0]}"
        
        # Remove None values and ensure we have valid compounds
        compounds = {k: v for k, v in compounds.items() if v and re.match(r'C[1-5]', v)}
        
    except Exception as e:
        print(f"⚠️ Error extracting tire compounds: {e}")
    
    return compounds


def extract_circuit_length(soup: BeautifulSoup) -> Optional[float]:
    """
    Extract circuit length from Pirelli race pages.
    
    Args:
        soup: Parsed HTML content
        
    Returns:
        Circuit length in km or None
    """
    try:
        page_text = soup.get_text()
        
        # Enhanced patterns for circuit length
        length_patterns = [
            r'(\d+\.?\d*)\s*km',  # "4.381 km"
            r'(\d+,\d+)\s*km',    # "4,381 km" (European format)
            r'length[:\s]*(\d+\.?\d*)',  # "Length: 4.381"
            r'distance[:\s]*(\d+\.?\d*)', # "Distance: 4.381"
            r'circuit[:\s]*(\d+\.?\d*)',  # "Circuit: 4.381"
            r'track[:\s]*(\d+\.?\d*)'     # "Track: 4.381"
        ]
        
        for pattern in length_patterns:
            matches = re.findall(pattern, page_text, re.IGNORECASE)
            if matches:
                try:
                    # Convert comma to dot for European number format
                    length_str = matches[0].replace(',', '.')
                    length = float(length_str)
                    # Reasonable validation (F1 tracks are typically 3-7 km)
                    if 2.0 <= length <= 8.0:
                        return length
                except ValueError:
                    continue
        
        # Look in meta tags or structured data
        meta_tags = soup.find_all('meta')
        for meta in meta_tags:
            content = meta.get('content', '')
            if 'km' in content:
                matches = re.findall(r'(\d+\.?\d*)\s*km', content)
                if matches:
                    try:
                        length = float(matches[0])
                        if 2.0 <= length <= 8.0:
                            return length
                    except ValueError:
                        continue
                
    except Exception as e:
        print(f"⚠️ Error extracting circuit length: {e}")
    
    return None


def extract_track_characteristics(soup: BeautifulSoup) -> Dict[str, Optional[int]]:
    """
    Extract track characteristics (1-5 scale ratings) from Pirelli pages.
    
    Args:
        soup: Parsed HTML content
        
    Returns:
        Dictionary with track characteristic ratings
    """
    characteristics = {
        'traction': None,
        'asphalt_grip': None,
        'tire_stress': None,
        'braking': None,
        'lateral': None,
        'downforce': None,
        'asphalt_abrasion': None,
        'track_evolution': None
    }
    
    try:
        page_text = soup.get_text().lower()
        
        # Enhanced patterns for track characteristics
        # These patterns look for the characteristic name followed by a rating
        patterns = {
            'traction': [r'traction[:\s\-]*([1-5])', r'([1-5])[:\s\-]*traction'],
            'asphalt_grip': [r'(?:asphalt\s*grip|grip)[:\s\-]*([1-5])', r'([1-5])[:\s\-]*(?:asphalt\s*grip|grip)'],
            'tire_stress': [r'(?:tire\s*stress|tyre\s*stress|stress)[:\s\-]*([1-5])', r'([1-5])[:\s\-]*(?:tire\s*stress|tyre\s*stress|stress)'],
            'braking': [r'braking[:\s\-]*([1-5])', r'([1-5])[:\s\-]*braking'],
            'lateral': [r'lateral[:\s\-]*([1-5])', r'([1-5])[:\s\-]*lateral'],
            'downforce': [r'downforce[:\s\-]*([1-5])', r'([1-5])[:\s\-]*downforce'],
            'asphalt_abrasion': [r'(?:asphalt\s*abrasion|abrasion)[:\s\-]*([1-5])', r'([1-5])[:\s\-]*(?:asphalt\s*abrasion|abrasion)'],
            'track_evolution': [r'(?:track\s*evolution|evolution)[:\s\-]*([1-5])', r'([1-5])[:\s\-]*(?:track\s*evolution|evolution)']
        }
        
        for char_name, pattern_list in patterns.items():
            for pattern in pattern_list:
                matches = re.findall(pattern, page_text)
                if matches:
                    try:
                        rating = int(matches[0])
                        if 1 <= rating <= 5:  # Validate rating is in expected range
                            characteristics[char_name] = rating
                            break  # Found valid rating, move to next characteristic
                    except ValueError:
                        continue
        
        # Look for characteristics in structured formats (tables, lists)
        tables = soup.find_all('table')
        for table in tables:
            rows = table.find_all('tr')
            for row in rows:
                row_text = row.get_text().lower()
                for char_name, pattern_list in patterns.items():
                    if characteristics[char_name] is None:  # Only if not found yet
                        for pattern in pattern_list:
                            matches = re.findall(pattern, row_text)
                            if matches:
                                try:
                                    rating = int(matches[0])
                                    if 1 <= rating <= 5:
                                        characteristics[char_name] = rating
                                        break
                                except ValueError:
                                    continue
        
        # Look in definition lists
        dl_elements = soup.find_all('dl')
        for dl in dl_elements:
            dl_text = dl.get_text().lower()
            for char_name, pattern_list in patterns.items():
                if characteristics[char_name] is None:
                    for pattern in pattern_list:
                        matches = re.findall(pattern, dl_text)
                        if matches:
                            try:
                                rating = int(matches[0])
                                if 1 <= rating <= 5:
                                    characteristics[char_name] = rating
                                    break
                            except ValueError:
                                continue
                
    except Exception as e:
        print(f"⚠️ Error extracting track characteristics: {e}")
    
    return characteristics


def get_race_links_for_year(year: int) -> List[str]:
    """
    Extract F1 race infographic links using the discovered pattern.
    
    Pattern: https://www.pirelli.com/global/en-ww/emotions-and-numbers/[race-name]-YEAR-[id]/
    
    Args:
        year: Year to scrape (2022-2024)
        
    Returns:
        List of race page URLs
    """
    year_url = f"https://www.pirelli.com/global/en-ww/emotions-and-numbers/infographics-{year}/"
    soup = get_page_content(year_url)
    
    if not soup:
        return []
    
    race_links = []
    
    try:
        print(f"🔍 Extracting F1 race links for {year}...")
        
        # Find all links that:
        # 1. Contain the year in the href
        # 2. Have an associated image (indicating infographic)
        # 3. Match the emotions-and-numbers pattern
        # 4. Contain F1-related terms
        
        all_links = soup.find_all('a', href=True)
        
        for link in all_links:
            href = link.get('href', '')
            
            # Check if this matches our pattern
            if (str(year) in href and 
                link.find('img') and 
                'emotions-and-numbers' in href and
                any(term in href.lower() for term in ['grand-prix', 'gp', 'formula-1', 'bahrain', 'australia', 
                                                     'japan', 'china', 'miami', 'spain', 'monaco', 'canada',
                                                     'austria', 'britain', 'hungary', 'belgium', 'netherlands',
                                                     'italy', 'singapore', 'qatar', 'usa', 'mexico', 'brazil',
                                                     'vegas', 'abu-dhabi', 'saudi', 'azerbaijan', 'imola'])):
                
                # Ensure it's a full URL
                if href.startswith('http'):
                    full_url = href
                else:
                    full_url = urljoin(year_url, href)
                
                if full_url not in race_links:
                    race_links.append(full_url)
                    # Extract race name from URL for display
                    race_name = href.split('/')[-2] if href.endswith('/') else href.split('/')[-1]
                    race_name = race_name.replace('-', ' ').title()
                    print(f"  🏁 {len(race_links):2d}. {race_name}")
        
        print(f"\n✅ Successfully extracted {len(race_links)} race links for {year}")
        
        # Validate we have a reasonable number (F1 typically has 20-24 races)
        if len(race_links) < 15:
            print(f"⚠️  Warning: Only found {len(race_links)} races. Expected 20-24 for a full F1 season.")
        elif len(race_links) > 25:
            print(f"⚠️  Warning: Found {len(race_links)} races. This seems high for F1.")
        else:
            print(f"✅ Race count looks good for F1 season!")
        
        return race_links
        
    except Exception as e:
        print(f"❌ Error extracting race links for {year}: {e}")
        return []


print("✅ Enhanced extraction functions loaded!")

✅ Enhanced extraction functions loaded!


## Main Scraping Function

In [21]:
def scrape_race_data(race_url: str, year: int) -> Optional[Dict]:
    """
    Scrape all tire and circuit data from a single race page.
    
    Args:
        race_url: URL of the race page
        year: Year of the race
        
    Returns:
        Dictionary with all extracted data or None if failed
    """
    soup = get_page_content(race_url)
    if not soup:
        return None
    
    try:
        # Extract race name from URL or page title
        race_name = "Unknown"
        title_element = soup.find('title')
        if title_element:
            title_text = title_element.get_text()
            # Clean up the title to get race name
            race_name = title_text.split('|')[0].strip() if '|' in title_text else title_text.strip()
        
        # Extract all data
        compounds = extract_tire_compounds(soup)
        circuit_length = extract_circuit_length(soup)
        characteristics = extract_track_characteristics(soup)
        
        # Combine all data
        race_data = {
            'year': year,
            'race_name': race_name,
            'url': race_url,
            'circuit_length_km': circuit_length,
            'soft_compound': compounds['soft'],
            'medium_compound': compounds['medium'],
            'hard_compound': compounds['hard'],
            **characteristics  # Unpack all track characteristics
        }
        
        print(f"✅ Extracted data for: {race_name} ({year})")
        return race_data
        
    except Exception as e:
        print(f"❌ Error scraping race data from {race_url}: {e}")
        return None


def scrape_all_pirelli_data(years: List[int] = YEARS) -> pd.DataFrame:
    """
    Scrape tire and circuit data for all races across specified years.
    
    Args:
        years: List of years to scrape
        
    Returns:
        DataFrame with all race data
    """
    all_race_data = []
    
    print(f"🚀 Starting Pirelli data scraping for years: {years}")
    print(f"📊 Target data: Tire compounds + 8 track characteristics")
    print("="*50)
    
    for year in years:
        print(f"\n📅 Processing year: {year}")
        
        # Get all race links for this year
        race_links = get_race_links_for_year(year)
        
        if not race_links:
            print(f"⚠️ No race links found for {year}")
            continue
        
        # Scrape each race
        for i, race_url in enumerate(race_links, 1):
            print(f"\n🏁 Race {i}/{len(race_links)} for {year}")
            
            race_data = scrape_race_data(race_url, year)
            if race_data:
                all_race_data.append(race_data)
            
            # Extra delay between races
            time.sleep(DELAY_BETWEEN_REQUESTS)
    
    print("\n" + "="*50)
    print(f"🎉 Scraping complete! Collected data for {len(all_race_data)} races")
    
    # Convert to DataFrame
    df = pd.DataFrame(all_race_data)
    
    if not df.empty:
        print(f"📊 DataFrame shape: {df.shape}")
        print(f"📋 Columns: {list(df.columns)}")
    
    return df


print("✅ Main scraping functions loaded!")

✅ Main scraping functions loaded!


## Test Single Page First

In [22]:
# Test the scraper on the Hungarian GP page to understand the structure
print("🧪 Testing scraper on Hungarian GP 2024 page...")

test_url = "https://www.pirelli.com/global/en-ww/emotions-and-numbers/formula-1-hungarian-grand-prix-2024-136985/"
test_soup = get_page_content(test_url)

if test_soup:
    print("✅ Successfully loaded Hungarian GP page")
    print(f"📄 Page title: {test_soup.find('title').get_text() if test_soup.find('title') else 'No title found'}")
    
    # Look for infographic images and their alt text
    print("\n🖼️ Infographic images found:")
    images = test_soup.find_all('img')
    for i, img in enumerate(images):
        src = img.get('src', '')
        alt = img.get('alt', '')
        if 'infographic' in src.lower() or 'info' in alt.lower() or any(keyword in alt.lower() for keyword in ['tire', 'compound', 'circuit', 'track']):
            print(f"  Image {i+1}: {alt} -> {src}")
    
    # Look for tire compound mentions (C1-C5)
    print("\n🛞 Tire compound references:")
    page_text = test_soup.get_text()
    compound_matches = re.findall(r'C[1-5]', page_text)
    print(f"Found compounds: {set(compound_matches)}")
    
    # Look for circuit length
    print("\n📏 Circuit length references:")
    length_matches = re.findall(r'(\d+\.?\d*)\s*km', page_text, re.IGNORECASE)
    print(f"Found lengths: {length_matches}")
    
    # Look for characteristic ratings (numbers 1-5)
    print("\n📊 Potential ratings (1-5 scale):")
    # Look for patterns like "Traction: 3" or similar
    rating_patterns = [
        r'traction[:\s]*([1-5])',
        r'grip[:\s]*([1-5])',
        r'stress[:\s]*([1-5])',
        r'braking[:\s]*([1-5])',
        r'lateral[:\s]*([1-5])',
        r'downforce[:\s]*([1-5])',
        r'abrasion[:\s]*([1-5])',
        r'evolution[:\s]*([1-5])'
    ]
    
    for pattern in rating_patterns:
        matches = re.findall(pattern, page_text.lower())
        if matches:
            print(f"  {pattern.split('[')[0]}: {matches}")
    
    # Look for structured data (JSON-LD, data attributes, etc.)
    print("\n📋 Structured data:")
    json_scripts = test_soup.find_all('script', type='application/ld+json')
    print(f"Found {len(json_scripts)} JSON-LD scripts")
    
    # Look for data attributes
    elements_with_data = test_soup.find_all(attrs={"data-compound": True})
    if elements_with_data:
        print(f"Found {len(elements_with_data)} elements with data-compound attributes")
    
    # Show some sample text content
    print("\n📝 Sample text content (first 1000 chars):")
    clean_text = ' '.join(page_text.split())  # Clean up whitespace
    print(clean_text[:1000])
    
else:
    print("❌ Failed to load Hungarian GP page")
    print("💡 Check URL and network connection")

🧪 Testing scraper on Hungarian GP 2024 page...
📡 Fetching: https://www.pirelli.com/global/en-ww/emotions-and-numbers/formula-1-hungarian-grand-prix-2024-136985/
✅ Successfully loaded Hungarian GP page
📄 Page title: FORMULA 1  HUNGARIAN  GRAND PRIX 2024 | Pirelli

🖼️ Infographic images found:

🛞 Tire compound references:
Found compounds: set()

📏 Circuit length references:
Found lengths: []

📊 Potential ratings (1-5 scale):

📋 Structured data:
Found 1 JSON-LD scripts

📝 Sample text content (first 1000 chars):
FORMULA 1 HUNGARIAN GRAND PRIX 2024 | Pirelli IT EN ES BR DE FR 中国 Stories Stories Road Racing Spot Life Road overview Car Motorcycles Bicycles Racing Spot overview Formula 1 Rally Gran Turismo Superbike Sailing Cycling Other Competitions E-sport Life overview Sustainability People Pirelli Calendar Lifestyle Innovation back Products Products Car Tyres Moto Tyres Bike Tyres Car Tyres overview Moto Tyres overview Bike Tyres overview back Lifestyle Lifestyle Pirelli Podium Cap Special

## Run Full Scraper

**⚠️ Important Notes:**
- This will make many requests to Pirelli's website
- The scraper includes delays to be respectful
- You may need to adjust the extraction functions based on the actual HTML structure
- Run the test cell above first to verify the scraper works

In [ ]:
# Run the full scraper (uncomment when ready)
# pirelli_data = scrape_all_pirelli_data()

# For now, let's start with just one year to test
print("🎯 Starting with 2024 data only for testing...")
pirelli_data = scrape_all_pirelli_data([2024])

# Display results
if not pirelli_data.empty:
    print("\n📊 Scraped Data Summary:")
    print(pirelli_data.head())
    
    print("\n📈 Data Info:")
    print(pirelli_data.info())
    
    print(f"\n✅ Successfully scraped {len(pirelli_data)} races")
    
else:
    print("❌ No data scraped")
    print("💡 This is expected since the data extraction functions need refinement for the infographic format")

🎯 Starting with 2024 data only for testing...
🚀 Starting Pirelli data scraping for years: [2024]
📊 Target data: Tire compounds + 8 track characteristics

📅 Processing year: 2024
📡 Fetching: https://www.pirelli.com/global/en-ww/emotions-and-numbers/infographics-2024/
🔍 Extracting F1 race links for 2024...
  🏁  1. Formula 1 Gulf Air Bahrain Grand Prix 2024 120834
  🏁  2. Formula 1 Rolex Australian Grand Prix 2024 123044
  🏁  3. Gp Japan2024 124711
  🏁  4. Gp China2024 127209
  🏁  5. Formula 1 Crypto Com Miami Grand Prix 2024 128694
  🏁  6. Formula 1 Msc Cruises Gran Premio Del Made In Italy E Dell Emilia Romagna 2024 129842
  🏁  7. Formula 1 Grand Prix De Monaco 2024 130812
  🏁  8. Formula 1 Pirelli Grand Prix Du Canada 2024 131658
  🏁  9. Formula 1 Aramco Gran Premio De Espana 2024 133752
  🏁 10. Formula 1 Qatar Airways Austrian Grand Prix 2024 134873
  🏁 11. Formula 1 Qatar Airways British Grand Prix 2024 135487
  🏁 12. Formula 1 Hungarian Grand Prix 2024 136985
  🏁 13. Formula 1 Rolex

## ✅ Current Status

**Race Link Extraction: WORKING** 
- Successfully extracting 23 race links for 2024 (correct number for F1 season)
- Pattern identified: `<a href="...race..."><img ...></a>` with year in URL
- Function `get_race_links_for_year()` is working properly

**Next Steps:**
- The race links are working perfectly
- Data extraction from individual race pages needs refinement (tire compounds, circuit characteristics)
- The infographic data is likely embedded in images rather than text

In [24]:
# Debug: Fetch and analyze the main Pirelli page structure
debug_url = "https://www.pirelli.com/global/en-ww/emotions-and-numbers/infographics-2024/"
print(f"🔍 Debugging main page: {debug_url}")

try:
    debug_response = requests.get(debug_url, headers=HEADERS, timeout=30)
    print(f"📊 Response status: {debug_response.status_code}")
    
    if debug_response.status_code == 200:
        debug_soup = BeautifulSoup(debug_response.content, 'html.parser')
        
        # Look for all links on the page
        all_links = debug_soup.find_all('a', href=True)
        print(f"🔗 Total links found: {len(all_links)}")
        
        # Look for links that might contain race information
        race_keywords = ['grand-prix', 'gp', 'race', 'circuit', 'track', 'bahrain', 'saudi', 'australia', 'japan', 'china', 'miami', 'imola', 'monaco', 'canada', 'spain', 'austria', 'britain', 'hungary', 'belgium', 'netherlands', 'italy', 'singapore', 'usa', 'mexico', 'brazil', 'vegas', 'qatar', 'abu-dhabi']
        
        potential_race_links = []
        for link in all_links:
            href = link.get('href', '').lower()
            text = link.get_text(strip=True).lower()
            
            # Check if any race keywords are in the href or text
            if any(keyword in href or keyword in text for keyword in race_keywords):
                potential_race_links.append({
                    'href': link.get('href'),
                    'text': link.get_text(strip=True),
                    'full_url': urljoin(debug_url, link.get('href'))
                })
        
        print(f"🏁 Potential race links found: {len(potential_race_links)}")
        
        # Show first few potential race links
        for i, link in enumerate(potential_race_links[:10]):
            print(f"  {i+1}. Text: '{link['text']}'")
            print(f"     URL: {link['full_url']}")
            print()
            
        # Also check for any links with infographics in the URL
        infographic_links = [link for link in all_links if 'infographic' in link.get('href', '').lower()]
        print(f"📈 Infographic links found: {len(infographic_links)}")
        
        for i, link in enumerate(infographic_links[:5]):
            print(f"  {i+1}. Text: '{link.get_text(strip=True)}'")
            print(f"     URL: {urljoin(debug_url, link.get('href'))}")
            
    else:
        print(f"❌ Failed to fetch page: {debug_response.status_code}")
        
except Exception as e:
    print(f"❌ Error debugging page: {e}")

🔍 Debugging main page: https://www.pirelli.com/global/en-ww/emotions-and-numbers/infographics-2024/
📊 Response status: 200
🔗 Total links found: 256
🏁 Potential race links found: 35
  1. Text: 'Racing Spot'
     URL: https://www.pirelli.com/global/en-ww/race/racingspot/

  2. Text: 'Racing Spot overview'
     URL: https://www.pirelli.com/global/en-ww/race/racingspot/

  3. Text: 'Formula 1'
     URL: https://www.pirelli.com/global/en-ww/race/racingspot/formula-1/

  4. Text: 'Rally'
     URL: https://www.pirelli.com/global/en-ww/race/racingspot/rally/

  5. Text: 'Gran Turismo'
     URL: https://www.pirelli.com/global/en-ww/race/racingspot/gran-turismo/

  6. Text: 'Superbike'
     URL: https://www.pirelli.com/global/en-ww/race/racingspot/superbike/

  7. Text: 'Sailing'
     URL: https://www.pirelli.com/global/en-ww/race/racingspot/sailing/

  8. Text: 'Cycling'
     URL: https://www.pirelli.com/global/en-ww/race/racingspot/cycling/

  9. Text: 'Other Competitions'
     URL: https://ww

In [25]:
# Deeper analysis of the page content
print("🔍 Looking for specific content patterns...")

# Look for any mention of race names or F1 circuits
page_text = debug_soup.get_text().lower()
race_names = ['bahrain', 'saudi arabia', 'australia', 'japan', 'china', 'miami', 'imola', 'monaco', 'canada', 'spain', 'austria', 'britain', 'hungary', 'belgium', 'netherlands', 'italy', 'singapore', 'usa', 'mexico', 'brazil', 'las vegas', 'qatar', 'abu dhabi']

found_races = []
for race in race_names:
    if race in page_text:
        found_races.append(race)

print(f"🏁 Race names found in page text: {found_races}")

# Look for script tags that might contain dynamic content
scripts = debug_soup.find_all('script')
print(f"📜 Script tags found: {len(scripts)}")

# Check for any JSON or data structures in scripts
for i, script in enumerate(scripts[:5]):  # Check first 5 scripts
    if script.string:
        content = script.string.strip()
        if len(content) > 0:
            print(f"\n📜 Script {i+1} content preview (first 200 chars):")
            print(content[:200] + "..." if len(content) > 200 else content)

# Look for div/section tags that might contain race data
race_containers = debug_soup.find_all(['div', 'section'], class_=True)
relevant_containers = []

for container in race_containers:
    class_names = ' '.join(container.get('class', []))
    if any(keyword in class_names.lower() for keyword in ['race', 'grand', 'prix', 'circuit', 'infographic']):
        relevant_containers.append({
            'tag': container.name,
            'classes': class_names,
            'text_preview': container.get_text(strip=True)[:100]
        })

print(f"\n🎯 Relevant containers found: {len(relevant_containers)}")
for i, container in enumerate(relevant_containers[:5]):
    print(f"  {i+1}. Tag: {container['tag']}, Classes: {container['classes']}")
    print(f"     Preview: {container['text_preview']}")
    print()

🔍 Looking for specific content patterns...
🏁 Race names found in page text: []
📜 Script tags found: 11

📜 Script 2 content preview (first 200 chars):
if (
    (!window.cookieBarAdded || window.cookieBarAdded === undefined) &&
    window.location.host !== "kolumbus-pirelli-preprod.dshare.cloud"
  ) {
    analyticsEvent = function () {};
    analytic...

📜 Script 3 content preview (first 200 chars):
/* <![CDATA[ */
  var google_conversion_id = 882404798;
  var google_custom_params = window.google_tag_params;
  var google_remarketing_only = true;
  /* ]]> */

📜 Script 5 content preview (first 200 chars):
<div class="lightbox-wrapper">
    <div class="over-lightbox"></div>
    <div class="lightbox">
      <a title="Close" class="close-lightbox" href="javascript:void(0)">&times;</a>
      <div class="co...

🎯 Relevant containers found: 0


## Save Data to CSV

In [26]:
# Save the data to CSV for use in your tire degradation analysis
if not pirelli_data.empty:
    output_file = "pirelli_tire_circuit_data.csv"
    pirelli_data.to_csv(output_file, index=False)
    
    print(f"💾 Data saved to: {output_file}")
    print(f"📊 Shape: {pirelli_data.shape}")
    print(f"📋 Columns: {list(pirelli_data.columns)}")
    
    # Show data quality summary
    print("\n🔍 Data Quality Summary:")
    missing_data = pirelli_data.isnull().sum()
    print(missing_data[missing_data > 0])
    
    # Show unique tire compounds found
    print("\n🛞 Tire Compounds Found:")
    for compound_type in ['soft_compound', 'medium_compound', 'hard_compound']:
        if compound_type in pirelli_data.columns:
            unique_compounds = pirelli_data[compound_type].dropna().unique()
            print(f"  {compound_type}: {unique_compounds}")
    
else:
    print("❌ No data to save")

❌ No data to save


## Integration with F1 Analysis

Once you have the Pirelli data, you can merge it with your FastF1 analysis:

In [27]:
# Example of how to integrate with your F1 tire degradation analysis
if not pirelli_data.empty:
    print("🔗 Integration Example:")
    print("""# In your F1 analysis notebook:
    
import pandas as pd
import fastf1

# Load Pirelli data
pirelli_data = pd.read_csv('pirelli_tire_circuit_data.csv')

# Load F1 session
session = fastf1.get_session(2024, "Bahrain", "R")
session.load()

# Process with your functions
processed_laps = process_race_for_tire_analysis(session)

# Merge with circuit characteristics
race_name = "Bahrain Grand Prix"  # Match with pirelli_data
circuit_data = pirelli_data[pirelli_data['race_name'].str.contains('Bahrain')]

if not circuit_data.empty:
    # Add circuit characteristics to your lap data
    for col in ['traction', 'asphalt_grip', 'tire_stress', 'braking']:
        if col in circuit_data.columns:
            processed_laps[f'circuit_{col}'] = circuit_data[col].iloc[0]

# Now you have lap-by-lap data with official circuit characteristics!
# Perfect for advanced tire degradation modeling
""")
    
    print("\n🎯 Benefits:")
    print("✅ Official tire compound data (C1-C5)")
    print("✅ Circuit characteristics for modeling")
    print("✅ Track evolution and abrasion data")
    print("✅ Braking and lateral force intensity")
    print("✅ Perfect complement to your FastF1 analysis!")

else:
    print("❌ No data available for integration example")

❌ No data available for integration example


## Troubleshooting and Refinement

If the scraper doesn't work perfectly on the first try, here are debugging tools:

In [28]:
# Enhanced debug function for Pirelli pages
def debug_pirelli_race_page(race_url: str):
    """
    Enhanced debugging specifically for Pirelli race pages.
    """
    print(f"🔍 Deep analysis of: {race_url}")
    
    soup = get_page_content(race_url)
    if not soup:
        return None
    
    print("\n📄 Page title:")
    title = soup.find('title')
    print(title.get_text() if title else "No title found")
    
    # Look for the main content area
    print("\n🎯 Main content structure:")
    main_content = soup.find('main') or soup.find('div', class_=re.compile(r'content|main'))
    if main_content:
        print("✅ Found main content area")
    
    # Look for infographic containers
    print("\n🖼️ Infographic analysis:")
    # Common patterns for infographic containers
    infographic_selectors = [
        {'class': re.compile(r'infographic', re.I)},
        {'class': re.compile(r'chart', re.I)},
        {'class': re.compile(r'data', re.I)},
        {'id': re.compile(r'infographic', re.I)}
    ]
    
    infographic_containers = []
    for selector in infographic_selectors:
        containers = soup.find_all('div', selector)
        infographic_containers.extend(containers)
    
    print(f"Found {len(infographic_containers)} potential infographic containers")
    
    # Analyze images more thoroughly
    images = soup.find_all('img')
    tire_related_images = []
    for img in images:
        src = img.get('src', '').lower()
        alt = img.get('alt', '').lower()
        
        if any(keyword in src + alt for keyword in ['tire', 'tyre', 'compound', 'circuit', 'track', 'infographic']):
            tire_related_images.append(img)
            print(f"  🎯 Relevant image: {alt[:50]} | {src}")
    
    # Look for tire compound data in various formats
    print(f"\n🛞 Tire compound extraction:")
    page_text = soup.get_text()
    
    # Enhanced compound detection
    compound_patterns = [
        r'soft[:\s]*C([1-5])',
        r'medium[:\s]*C([1-5])',
        r'hard[:\s]*C([1-5])',
        r'C([1-5])[:\s]*soft',
        r'C([1-5])[:\s]*medium',
        r'C([1-5])[:\s]*hard'
    ]
    
    compounds_found = {}
    for pattern in compound_patterns:
        matches = re.findall(pattern, page_text, re.IGNORECASE)
        if matches:
            compound_type = pattern.split('[')[0].replace('C([1-5])[:\\s]*', '').replace('[:\\s]*C([1-5])', '')
            compounds_found[compound_type] = matches
    
    print(f"Compound patterns found: {compounds_found}")
    
    # Look for circuit characteristics in structured format
    print(f"\n📊 Track characteristics analysis:")
    
    # Try to find characteristics in tables or structured lists
    tables = soup.find_all('table')
    for i, table in enumerate(tables):
        table_text = table.get_text().lower()
        if any(keyword in table_text for keyword in ['traction', 'grip', 'braking', 'stress']):
            print(f"  📋 Relevant table {i+1} found")
            rows = table.find_all('tr')
            for row in rows[:5]:  # Show first 5 rows
                print(f"    {row.get_text().strip()}")
    
    # Look for definition lists or structured data
    dl_elements = soup.find_all('dl')  # Definition lists
    for dl in dl_elements:
        dl_text = dl.get_text().lower()
        if any(keyword in dl_text for keyword in ['traction', 'grip', 'braking']):
            print(f"  📝 Definition list with characteristics found")
            break
    
    # Look for circuit length in multiple formats
    print(f"\n📏 Circuit length detection:")
    length_patterns = [
        r'(\d+\.?\d*)\s*km',
        r'(\d+,\d+)\s*km',
        r'length[:\s]*(\d+\.?\d*)',
        r'distance[:\s]*(\d+\.?\d*)'
    ]
    
    for pattern in length_patterns:
        matches = re.findall(pattern, page_text, re.IGNORECASE)
        if matches:
            print(f"  Pattern '{pattern}': {matches}")
    
    # Return the soup for further analysis
    return soup

# Test the enhanced debug function
print("🔧 Enhanced debug function ready!")
print("📝 Run: debug_pirelli_race_page('URL') to analyze a specific page")

# Automatically run on Hungarian GP
hungarian_soup = debug_pirelli_race_page("https://www.pirelli.com/global/en-ww/emotions-and-numbers/formula-1-hungarian-grand-prix-2024-136985/")

🔧 Enhanced debug function ready!
📝 Run: debug_pirelli_race_page('URL') to analyze a specific page
🔍 Deep analysis of: https://www.pirelli.com/global/en-ww/emotions-and-numbers/formula-1-hungarian-grand-prix-2024-136985/
📡 Fetching: https://www.pirelli.com/global/en-ww/emotions-and-numbers/formula-1-hungarian-grand-prix-2024-136985/

📄 Page title:
FORMULA 1  HUNGARIAN  GRAND PRIX 2024 | Pirelli

🎯 Main content structure:
✅ Found main content area

🖼️ Infographic analysis:
Found 2 potential infographic containers

🛞 Tire compound extraction:
Compound patterns found: {}

📊 Track characteristics analysis:

📏 Circuit length detection:

📄 Page title:
FORMULA 1  HUNGARIAN  GRAND PRIX 2024 | Pirelli

🎯 Main content structure:
✅ Found main content area

🖼️ Infographic analysis:
Found 2 potential infographic containers

🛞 Tire compound extraction:
Compound patterns found: {}

📊 Track characteristics analysis:

📏 Circuit length detection:


In [39]:
# Test the race link extraction
print("🧪 Testing race link extraction for 2024...")
test_2024_links = get_race_links_for_year(2024)

print(f"\n📊 Summary:")
print(f"Total 2024 race links found: {len(test_2024_links)}")

if test_2024_links:
    print(f"\nFirst 5 links:")
    for i, link in enumerate(test_2024_links[:5]):
        print(f"  {i+1}. {link}")
    
    if len(test_2024_links) > 5:
        print(f"  ... and {len(test_2024_links) - 5} more")
        
    # Test one link accessibility
    print(f"\n🧪 Testing first race link accessibility...")
    test_soup = get_page_content(test_2024_links[0])
    if test_soup:
        title = test_soup.find('title')
        print(f"✅ Successfully accessed: {title.get_text() if title else 'No title found'}")
    else:
        print("❌ Could not access the race page")
else:
    print("❌ No race links found")

🧪 Testing race link extraction for 2024...
📡 Fetching: https://www.pirelli.com/global/en-ww/emotions-and-numbers/infographics-2024/
🔍 Extracting F1 race links for 2024...
  🏁  1. Formula 1 Gulf Air Bahrain Grand Prix 2024 120834
  🏁  2. Formula 1 Rolex Australian Grand Prix 2024 123044
  🏁  3. Gp Japan2024 124711
  🏁  4. Gp China2024 127209
  🏁  5. Formula 1 Crypto Com Miami Grand Prix 2024 128694
  🏁  6. Formula 1 Msc Cruises Gran Premio Del Made In Italy E Dell Emilia Romagna 2024 129842
  🏁  7. Formula 1 Grand Prix De Monaco 2024 130812
  🏁  8. Formula 1 Pirelli Grand Prix Du Canada 2024 131658
  🏁  9. Formula 1 Aramco Gran Premio De Espana 2024 133752
  🏁 10. Formula 1 Qatar Airways Austrian Grand Prix 2024 134873
  🏁 11. Formula 1 Qatar Airways British Grand Prix 2024 135487
  🏁 12. Formula 1 Hungarian Grand Prix 2024 136985
  🏁 13. Formula 1 Rolex Belgian Grand Prix 2024 137042
  🏁 14. Formula 1 Heineken Dutch Grand Prix 2024 139980
  🏁 15. Formula 1 Pirelli Gran Premio D Italia 

## Summary

This notebook provides a comprehensive web scraper for Pirelli's F1 tire and circuit data.

### What it scrapes:
1. **Tire Compounds** - Official C1-C5 designations for soft/medium/hard
2. **Circuit Length** - Track distance in kilometers
3. **Track Characteristics** (1-5 scale):
   - Traction
   - Asphalt Grip
   - Tire Stress
   - Braking
   - Lateral Forces
   - Downforce
   - Asphalt Abrasion
   - Track Evolution

### Next Steps:
1. **Test** the scraper on a few pages first
2. **Refine** extraction functions based on actual HTML structure
3. **Run** full scraper for all years (2022-2024)
4. **Integrate** data with your F1 tire degradation analysis

### Integration with your analysis:
The scraped data perfectly complements your FastF1 analysis by providing:
- Official tire compound context
- Circuit-specific characteristics for modeling
- Track evolution and surface data
- Braking and cornering intensity metrics

This will make your tire degradation analysis much more comprehensive! 🏎️📊